## Passing Data Between Parent and Child Components

In the real life project, you would often like to reuse some components for example a numpad, which could be used whenever you want to put the number in your input field. In this scenario you will often have to pass the data from parent to child, or be able to modify the data in parent by using the child component methods.

In React, we passed the variables and functions from parent to child as a props. In Angular we use Input and Output decorators. Let's first create a parent component, where we have a variable we want to pass to child.

###### Remark: Below naming of Outputs is not according to conventions, but I wanted to show clearly what is happening in the code. Few words about the naming convention can be found at the end of this chapter.

In [ ]:
## parent.ts

import { Component } from '@angular/core';
import { Numpad } from "../numpad/numpad";
import { ChildComponent } from "../child-component/child-component";

@Component({
  selector: 'app-parent-component',
  imports: [ChildComponent],
  templateUrl: './parent-component.html',
  styleUrl: './parent-component.css',
})
export class ParentComponent {

  parentValue = 0;        ## <-- value we want to pass to child

  increaseParentValue() { ## <-- method we want to pass to child
    this.parentValue++
  }

  decreaseParentValue() { ## <--- method we want to pass to child
    this.parentValue--
  }

}

Now we have to set up the child component. If we want to "use" the parent variable/methods from parent we have to import Input and Output,and use it as in the following code:

In [ ]:
## child.ts

import { Component, EventEmitter, Input, Output} from '@angular/core';

@Component({
  selector: 'app-child-component',
  imports: [],
  templateUrl: './child-component.html',
  styleUrl: './child-component.css',
})
export class ChildComponent {

  @Input() valueFromParent = 0;  ## <-- this is how we initialize the value in child, this value will get the passed parent value
    
  @Output() increaseParentValue = new EventEmitter<number>() ## <-- this is an initialized Output emitter
  @Output() decreaseParentValue = new EventEmitter<number>() ## <-- this is an initialized Output emitter
  @Output() passPrivateValueToParent = new EventEmitter<number>() ## <-- this is an initialized Output emitter


  ## methods in parent to trigger the emitter (it's like you would invoke the method from parent)
  increaseValueInChild() {
    this.increaseParentValue.emit()
  }

  decreaseValueInChild() {

    this.decreaseParentValue.emit()
  }

 }


Now, we have to pass the required data from parent to child.

In [ ]:
## parent.html 

<h3>Parent value {{parentValue}}</h3>  ## <-- display parentValue (this value will change when modified by child methods)
<app-child-component
    ## passing variable to child: [inputNameFromChild]="variableNameFromParent"
    [valueFromParent]="parentValue" 
    ## passing method to child emitter: (emitterNameFromChild)="parentMethodFromParent"
    (increaseParentValue)="increaseParentValue()"
    (decreaseParentValue)="decreaseParentValue()"
    (passPrivateValueToParent)="receiveValueFromChild($event)"
>
</app-child-component>


## child.html

<h3>Parent value in child {{valueFromParent}}</h3>  ## <-- we can display the value from parent
<div class="flex gap-4">
    ## the click event triggers the method from child (method triggers emitter)
    <button class="btn btn-info" (click)="increaseValueInChild()">Child increase</button>
    <button class="btn btn-info" (click)="decreaseValueInChild()">Child decrease</button>
</div>

#### Passing data from child to parent

You can also modify the parent by passing some data from the child. Let's talk about the numpad again. We want to pass the selected number from numpad to the parent. We can do it as similar as in the below code.

In [ ]:
## parent.ts

import { Component } from '@angular/core';
import { Numpad } from "../numpad/numpad";
import { ChildComponent } from "../child-component/child-component";

@Component({
  selector: 'app-parent-component',
  imports: [ChildComponent],
  templateUrl: './parent-component.html',
  styleUrl: './parent-component.css',
})
export class ParentComponent {

  valueFromChild = 0  ## <-- we want to modify this variable based on the value from the child

  receiveValueFromChild(childValue: number): number { ## <-- method which will receive the data from child
    return this.valueFromChild = childValue
  }

}

## child.ts

import { Component, EventEmitter, Input, Output, OnInit, AfterViewInit } from '@angular/core';

@Component({
  selector: 'app-child-component',
  imports: [],
  templateUrl: './child-component.html',
  styleUrl: './child-component.css',
})
export class ChildComponent {

  @Input() valueFromParent = 0;
  @Output() increaseParentValue = new EventEmitter<number>()
  @Output() decreaseParentValue = new EventEmitter<number>()
  @Output() increaseParentControlledValue = new EventEmitter<number>()
  @Output() passPrivateValueToParent = new EventEmitter<number>()

  private childValue = 13  ## <-- we want to pass this value to parent

  ngAfterViewIniti() {
    this.passPrivateValueToParent.emit(this.childValue)
  }
    

  passValueFromChildToParent(): void {
    this.passPrivateValueToParent.emit(childValue) ## <-- we are emitting the childValue to parent
  }

  ## below example shows how we can pass the value as an arguments

  passValueFromChildToParent(num: number): void {
    this.passPrivateValueToParent.emit(num)
  }

 }


## Naming convention

##### Below naming convention regards the emitters name. 

Key rule (very important)

A child component must never talk about the parent in its API

Why?

The child should be reusable

The same component could be used in many parents

“parent” is an implementation detail

✅ Better alternatives (recommended)

If the value is generic

@Output() valueIncreased = new EventEmitter<number>();
    
@Output() valueDecreased = new EventEmitter<number>();

If the value is domain-specific (best)
    
@Output() diameterIncreased = new EventEmitter<number>();
    
@Output() diameterDecreased = new EventEmitter<number>();

If you want the cleanest API
    
@Output() valueChanged = new EventEmitter<number>();

❌ What Angular devs would reject in a code review
    
parentValueIncreased
    
increaseParentValue
    
callParent
    
sendToParent

## Name	Verdict
    
increaseParentValue	❌ Bad
    
parentValueIncreased	⚠️ Better, but still not ideal
    
valueIncreased	✅ Good
    
diameterChanged	✅ Best (for your case)